In [85]:
import glob
from json import JSONDecodeError

import pandas as pd
import os
import ast
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
#We will define a function which will store all scraped ads through api calls
#We will enrich the dataset with certain features which are in the ad rather than from the response of the api call

In [11]:
ads[0]['id']

41737621

In [6]:
init_car_df = pd.read_csv('../data/processed_ads_df.csv')

In [16]:
init_car_df.loc[init_car_df.id==41737621]

,lat,lon,Unnamed: 0,is_new,label,model_stub,thumbs,id,price_debatable,user_id,...,registration_year,battery_charge_time,battery_range,engine_power,fuel_type,gearbox_type,brand,variant,model,year
13878,40.652406,22.965476,f14a5e29-cc87-4597-bcaa-56f3067583b3,False,NaN,Opel Astra '08,"{'count': 11, 'urls': ['https://static.car.gr/...",41737621,False,1001043990,...,2008,NaN,NaN,96 bhp,Diesel,NaN,opel,NaN,astra,8


In [37]:
ids = [os.path.splitext(os.path.basename(id))[0] for id in list(glob.glob('../data/ads/*.json'))]
ids_in_frame = init_car_df.id.astype(str).values.tolist()

In [39]:
count = ~np.isin(ids,ids_in_frame)

In [41]:
count.sum()

np.int64(0)

In [68]:
ads = [json.load(open(x,'r',encoding='utf-8')) for x in list(glob.glob('../data/ads/*.json'))[:10]]

In [71]:
len(ads)

10

In [46]:
ads[0].keys()

dict_keys(['automatic_deletion_date', 'catalogue_shops', 'category_ids', 'certificates', 'condition', 'crashed', 'deleted_date', 'description', 'discount', 'external_certificates', 'external_link', 'extra_specifications', 'extras', 'finance', 'hidden_date', 'id', 'key_features', 'leasing', 'location', 'model_trims', 'offertype', 'other_classifieds', 'parts_for_vehicle', 'photos', 'photos_compact', 'price', 'price_check', 'price_per_unit', 'seo', 'seo_json_ld', 'short_videos', 'show_loans_info', 'similar_classifieds', 'similar_searches', 'specifications', 'states', 'tags', 'thumbs_patterns', 'title', 'title_parts', 'trades', 'videos'])

In [79]:
fields_to_keep = ['id','description','extras','specifications','model_trims']

In [51]:
ads[0]['extras'] # here we will need to get all names from every add and to get the unique names possible over the ads in order to have a set of all extras and treat it as a one hot encoding for each type of extra

[{'key': 412, 'name': 'abs', 'value': 'ABS'},
 {'key': 427, 'name': 'bluetooth', 'value': 'Bluetooth'},
 {'key': 437, 'name': 'cd_player', 'value': 'CD player'},
 {'key': 453, 'name': 'esp', 'value': 'ESP'},
 {'key': 459, 'name': 'immobilizer', 'value': 'Immobilizer'}]

In [252]:
ads[4]['specifications'] #Here we will need again

[{'label': 'Μάρκα - μοντέλο',
  'name': 'makemodel',
  'value': "Volkswagen Golf gt '08"},
 {'label': 'Νούμερο αγγελίας',
  'name': 'classified_number',
  'value': '343814526'},
 {'extra': {'debatable': True,
   'foreign_numbers': None,
   'raw_price': 7000,
   'without_vat': None},
  'label': 'Τιμή',
  'name': 'price',
  'raw_value': 7000,
  'value': '7.000 €'},
 {'label': 'Κατάσταση', 'name': 'condition', 'value': 'Μεταχειρισμένο'},
 {'label': 'Κατηγορία',
  'name': 'category',
  'value': [{'category': {'human_name_plural': 'Κουπέ-Σπόρ', 'id': 15215},
    'search': {'seo_url': '/classifieds/cars/?category=15'}}]},
 {'label': 'Χρονολογία', 'name': 'registration', 'value': '1/2008'},
 {'label': 'Χιλιόμετρα', 'name': 'mileage', 'value': '209.000 χλμ'},
 {'label': 'Καύσιμο', 'name': 'fuel_type', 'value': 'Βενζίνη'},
 {'label': 'Κυβικά', 'name': 'engine_size', 'value': '1.400 cc'},
 {'label': 'Ιπποδύναμη', 'name': 'engine_power', 'value': '180 bhp'},
 {'label': 'Σασμάν', 'name': 'gearbox_

In [109]:
ads[5]['model_trims'][0] #Make a function that if not empty extract fuel_consumption,

{'acceleration': 12.2,
 'body_type': 'hatchback',
 'doors': 5.0,
 'drive_type': 'FWD',
 'engine_power': 91.0,
 'engine_size': 999.0,
 'fuel_consumption': 4.9,
 'gearbox_type': 'manual',
 'gross_weight': 1603.0,
 'id': 74627,
 'make': 'Renault',
 'make_id': 14844,
 'model': 'Clio',
 'number_of_gears': 6.0,
 'top_speed': 180.0,
 'torque': 160.0,
 'trim': 'TCe 90 Experience',
 'vehicle_height': 1798.0,
 'vehicle_length': 4050.0,
 'vehicle_width': 1440.0,
 'wheelbase': 2583.0}

In [76]:
ads[6]['price_check']

{'current_price': 15500,
 'last_check': '2025-02-18 11:52:37',
 'market_price': 13000,
 'price_is_ok': True}

In [78]:
ads[6]['id']

43271412

In [86]:
def get_all_possible_extras(path_to_ads):
    all_extras = []
    for path in tqdm(path_to_ads):
        try:
            with open(path,'r',encoding='utf-8') as f:
                all_extras.extend([ad['name'] for ad in json.load(f)['extras']])
        except KeyError:
            continue
    return list(set(all_extras))


In [87]:
all_extras = get_all_possible_extras(list(glob.glob('../data/ads/*.json')))

100%|██████████| 113877/113877 [00:21<00:00, 5181.77it/s]


In [90]:
all_extras

['antique',
 'cheques_accepted',
 'turbo',
 'steering_lights',
 'collision_avoidance_system',
 'trip_computer',
 'wheelchair',
 'apple_carplay',
 'credit_card_accepted',
 'extra:827',
 'leather_seats',
 'anti_theft_system_gps',
 'rain_sensor',
 'panoramic_roof',
 'dvd',
 'automatic_parking',
 'warranty_extension',
 'from_organization_of_public_materials_management',
 'multi_purpose_steering_wheel',
 'tcs_asr',
 'tv_camera',
 'seats_7+',
 'power_windows',
 'bucket_seats',
 'hitch',
 'aircondition(a_c)',
 'heated_seats',
 'power_seats',
 'alumium_rims',
 'locking_differential',
 'android_auto',
 'accept_exchange_with_car',
 'hardtop',
 'right_hand_drive_car',
 'roof_rails',
 'armored',
 'lane_assist',
 'imported',
 'left_sliding_door',
 'agricultural',
 'never_crashed',
 'immobilizer',
 'notes_accepted',
 'rear_view_camera',
 'air_suspension',
 'bluetooth',
 'leasing',
 'price_without_vat',
 'parktronic',
 'withdrawal_right',
 'isofix_children_seats',
 'electric_sunroof',
 'abs',
 'telep

In [92]:
def get_all_possible_specifications(path_to_ads):
    all_specifications = []
    for path in tqdm(path_to_ads):
        try:
            with open(path, 'r', encoding='utf-8') as f:
                all_specifications.extend([spec['name'] for spec in json.load(f)['specifications']])
        except KeyError:
            continue
    return list(set(all_specifications))


In [93]:
all_specs = get_all_possible_specifications(list(glob.glob('../data/ads/*.json')))

100%|██████████| 113877/113877 [00:24<00:00, 4561.70it/s]


In [96]:
all_specs

['leasing',
 'fuel_type',
 'title',
 'multichild_price',
 'finance',
 'drive_type',
 'interior_type',
 'makemodel',
 'circulation_tax',
 'home_delivery',
 'mileage',
 'battery_range',
 'seats',
 'for_makemodels',
 'classified_number',
 'fuel_consumption',
 'condition',
 'kteo',
 'exterior_color',
 'gearbox_type',
 'rim_size',
 'category',
 'number_plate_ending',
 'emissions_co2',
 'battery_charge_time',
 'doors',
 'previous_owners',
 'airbags',
 'engine_size',
 'price',
 'euroclass',
 'battery_quick_charge_time',
 'offertype',
 'interior_color',
 'registration',
 'battery_capacity',
 'engine_power']

We will keep specs:


In [187]:
specs = ['drive_type', 'interior_type', 'battery_range', 'seats', 'fuel_consumption', 'kteo', 'exterior_color','number_plate_ending','emissions_co2','battery_charge_time', 'doors', 'interior_color', 'rim_size']

In [180]:
def get_all_possible_model_trims(path_to_ads):
    all_model_trims = []
    for path in tqdm(path_to_ads):
        try:
            with open(path, 'r', encoding='utf-8') as f:
                ad = json.load(f)['model_trims']
                if ad:
                    all_model_trims.extend([trim for trim in ad[0].keys()])
        except KeyError:
            continue
    return list(set(all_model_trims))


In [181]:
model_trims = get_all_possible_model_trims(list(glob.glob('../data/ads/*.json')))

100%|██████████| 113877/113877 [00:14<00:00, 7797.23it/s]


In [183]:
model_trims

['vehicle_height',
 'number_of_gears',
 'torque',
 'gross_weight',
 'acceleration',
 'vehicle_width',
 'drive_type',
 'id',
 'body_type',
 'model',
 'fuel_consumption',
 'make_id',
 'gearbox_type',
 'vehicle_length',
 'make',
 'top_speed',
 'doors',
 'engine_size',
 'trim',
 'wheelbase',
 'engine_power']

In [261]:
intersection = list(set(all_specs) & set(model_trims))

In [262]:
intersection

['doors',
 'drive_type',
 'engine_size',
 'gearbox_type',
 'fuel_consumption',
 'engine_power']

aModel trims we will keep:

In [184]:
model_trims_to_keep = ['vehicle_height',
 'number_of_gears',
 'torque',
 'gross_weight',
 'acceleration',
 'vehicle_width',
 'drive_type', #common with specs
 'body_type',
 'fuel_consumption', #common with specs
 'vehicle_length',
 'top_speed',
 'doors', #common with specs
 'trim',
 'wheelbase']

In [ ]:
#get ads with fields necessary

def load_json(path):
    try:
        with open(path,'r',encoding='utf-8') as f:
            ad = json.load(f)
            return {key: ad[key] for key in fields_to_keep if key in ad.keys()}
    except (JSONDecodeError, KeyError) as e:
        return None

So now we will define our processing method to create the enriching dataset based on the scraped ads:
For each ad:
* Extract description, extras, specifications and model_trims
* transform extras as a dict with all possible extras and true or false according to the presence of the key in the extras of the ad
* transform specifications as a dict with all possible kept specs and their values without doors, fuel_consumption, drive_type
* transform model trims as a dict with all possible kept specs and their values without doors, fuel_consumption, drive_type
* if doors, fuel_conumption, drive_type in model_trims create dict with the values else test in specs else return a dict with nan



In [331]:
def process_extras(extras, extras_to_keep):
    if isinstance(extras,list) and extras:
        present_extras = {extra['name'] for extra in extras}
        return {extra: (extra in present_extras) for extra in extras_to_keep}
    return {extra: None for extra in extras_to_keep}

def process_specifications(specs, specs_to_keep):
    if isinstance(specs,list) and specs:
        return {spec: next((item['value'] for item in specs if item['name'] == spec),None)  for spec in specs_to_keep}
    return {spec: None for spec in specs_to_keep}

def process_model_trims(model_trims, model_trims_to_keep):
    if model_trims:
        return {trim : model_trims[0][trim] for trim in model_trims_to_keep}
    return {trim: None for trim in model_trims_to_keep}

def process_fuel_consumption(specs,model_trims):
    if isinstance(model_trims,list) and model_trims and 'fuel_consumption' in model_trims[0].keys():
        return {'fuel_consumption': model_trims[0]['fuel_consumption']}
    else:
        res = next((item['value'] for item in specs if item['name'] == 'fuel_consumption'),None)
        if isinstance(res,dict) and 'mixed' in res and 'value' in res['mixed']:
            return {'fuel_consumption' : res['mixed']['value']}

    return {'fuel_consumption': None}


def process_drive_type(specs,model_trims):
    if isinstance(model_trims,list) and model_trims and 'drive_type' in model_trims[0].keys():
        return {'drive_type': model_trims[0]['drive_type']}

    return {'drive_type' : next((item['value'] for item in specs if item['name'] == 'drive_type'),None)}


def process_doors(specs,model_trims):
    if isinstance(model_trims,list) and model_trims and 'doors' in model_trims[0].keys():
        return {'doors': model_trims[0]['doors']}

    return {'doors' : next((item['value'] for item in specs if item['name'] == 'doors'),None)}

In [250]:
process_fuel_consumption(ads[4]['specifications'],ads[4]['model_trims'])

{'fuel_consumption': '7,50'}

In [258]:
process_drive_type(ads[8]['specifications'],ads[4]['model_trims'])

{'drive_type': None}

In [264]:
process_doors(ads[8]['specifications'],ads[4]['model_trims'])

{'doors': '2'}

In [236]:
process_fuel_consumption(ads[4]['specifications'],ads[4]['model_trims'])

In [194]:
process_extras(ads[6]['extras'],all_extras)

{'antique': False,
 'cheques_accepted': False,
 'turbo': False,
 'steering_lights': False,
 'collision_avoidance_system': True,
 'trip_computer': True,
 'wheelchair': False,
 'apple_carplay': False,
 'credit_card_accepted': False,
 'extra:827': False,
 'leather_seats': True,
 'anti_theft_system_gps': False,
 'rain_sensor': False,
 'panoramic_roof': False,
 'dvd': False,
 'automatic_parking': False,
 'warranty_extension': False,
 'from_organization_of_public_materials_management': False,
 'multi_purpose_steering_wheel': True,
 'tcs_asr': False,
 'tv_camera': True,
 'seats_7+': False,
 'power_windows': True,
 'bucket_seats': True,
 'hitch': False,
 'aircondition(a_c)': False,
 'heated_seats': False,
 'power_seats': False,
 'alumium_rims': True,
 'locking_differential': False,
 'android_auto': True,
 'accept_exchange_with_car': True,
 'hardtop': False,
 'right_hand_drive_car': False,
 'roof_rails': False,
 'armored': False,
 'lane_assist': True,
 'imported': False,
 'left_sliding_door': F

In [196]:
process_specifications(ads[6]['specifications'],specs)

{'drive_type': 'Προσθιοκίνητο (FWD)',
 'interior_type': 'Δέρμα',
 'battery_range': None,
 'seats': '5',
 'fuel_consumption': {'city': {'label': 'Εντός', 'value': '6,80'},
  'mixed': {'label': 'Μικτό', 'value': '5,40'},
  'motorway': {'label': 'Εκτός', 'value': '4,50'}},
 'kteo': '10/2025',
 'exterior_color': 'Μαύρο (Μεταλλικό)',
 'number_plate_ending': 'Μονή',
 'emissions_co2': '124 g/km',
 'battery_charge_time': None,
 'doors': '5',
 'interior_color': 'Γκρι',
 'rim_size': '16 ίντσες'}

In [288]:
process_model_trims(ads[1]['model_trims'],model_trims_to_keep)

{'vehicle_height': None,
 'number_of_gears': None,
 'torque': None,
 'gross_weight': None,
 'acceleration': None,
 'vehicle_width': None,
 'body_type': None,
 'vehicle_length': None,
 'top_speed': None,
 'trim': None,
 'wheelbase': None}

In [266]:
{**{'car':'zouzou'}, **{'model': 'fifi'}}

{'car': 'zouzou', 'model': 'fifi'}

In [294]:
def process_description(ad):
    description = ad['description']
    if description is None:
        return {'description': None}
    else:
        return {'description': description['content']}

In [314]:
def get_model_trims(ad):
    try :
        return ad['model_trims']
    except KeyError:
        return []

def get_specifications(ad):
    try :
        return ad['specifications']
    except KeyError:
        return []


def get_extras(ad):
    try :
        return ad['extras']
    except KeyError:
        return []

In [332]:
model_trims_to_keep = ['vehicle_height', 'number_of_gears', 'torque', 'gross_weight', 'acceleration', 'vehicle_width', 'body_type', 'vehicle_length', 'top_speed', 'trim', 'wheelbase']

specs_to_keep = ['interior_type', 'battery_range', 'seats', 'kteo', 'exterior_color','number_plate_ending','emissions_co2','battery_charge_time', 'interior_color', 'rim_size']


def create_enrichment_set(path_to_ads):
    res_df = []
    failed_ads = []
    for path in tqdm(path_to_ads):
        try:
            with open(path,'r',encoding='utf-8') as f:
                ad = json.load(f)
                id_dict = {'id' : str(os.path.splitext(os.path.basename(path))[0])}
                description_dict = process_description(ad)
                extras = get_extras(ad)
                specifications = get_specifications(ad)
                model_trims = get_model_trims(ad)
                extras_dict = process_extras(extras,all_extras)
                specifications_dict = process_specifications(specifications,specs_to_keep)
                model_trims_dict = process_model_trims(model_trims,model_trims_to_keep)
                fuel_consumption_dict = process_fuel_consumption(specifications,model_trims)
                drive_type_dict = process_drive_type(specifications,model_trims)
                doors_dict = process_doors(specifications,model_trims)
                res_df.append({**id_dict,**description_dict,**extras_dict,**specifications_dict,**model_trims_dict,**fuel_consumption_dict,**drive_type_dict,**doors_dict})
        except KeyError:
            failed_ads.append(path)
    return res_df,failed_ads



In [333]:
df,failed_ads = create_enrichment_set(list(glob.glob('../data/ads/*.json')))

100%|██████████| 113877/113877 [00:36<00:00, 3113.43it/s]


In [334]:
len(df)

113877

In [339]:
enrichment_df = pd.DataFrame(df)

In [365]:
enrichment_df.fuel_consumption

0         None
1         3,80
2         None
3          6.8
4         7,50
          ... 
113872    None
113873    None
113874     3.6
113875    None
113876    11.1
Name: fuel_consumption, Length: 113877, dtype: object

In [342]:
enrichment_df.to_csv('../data/enrichment_df.csv',index=False)

In [354]:
pd.set_option('display.max_columns',None)
pd.set_option("display.width", 1000)

In [355]:
enrichment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113877 entries, 0 to 113876
Columns: 111 entries, id to doors
dtypes: float64(9), object(102)
memory usage: 96.4+ MB


In [364]:
enrichment_df.isna().sum()

id                      0
description         23326
antique             25265
cheques_accepted    25265
turbo               25265
                    ...  
trim                55710
wheelbase           52345
fuel_consumption    44596
drive_type          15434
doors                   6
Length: 111, dtype: int64

In [371]:
ads[5]['extras']

[]

In [369]:
enrichment_df.head(10)

,id,description,antique,cheques_accepted,turbo,steering_lights,collision_avoidance_system,trip_computer,wheelchair,apple_carplay,credit_card_accepted,extra:827,leather_seats,anti_theft_system_gps,rain_sensor,panoramic_roof,dvd,automatic_parking,warranty_extension,from_organization_of_public_materials_management,multi_purpose_steering_wheel,tcs_asr,tv_camera,seats_7+,power_windows,bucket_seats,hitch,aircondition(a_c),heated_seats,power_seats,alumium_rims,locking_differential,android_auto,accept_exchange_with_car,hardtop,right_hand_drive_car,roof_rails,armored,lane_assist,imported,left_sliding_door,agricultural,never_crashed,immobilizer,notes_accepted,rear_view_camera,air_suspension,bluetooth,leasing,price_without_vat,parktronic,withdrawal_right,isofix_children_seats,electric_sunroof,abs,telephone,led_lights,gps,greek_dealership,automatic_air_conditioning,central_locking,hill_assist,tft_screen,cd_player,warranty,head_up_display,service_book,power_mirrors,eco_start_stop,alarm,taxfree_foreign_numbers,xenon,fog_lights,nonsmoker,power_steering,sliding_doors_x2,radio_player,acc,usb,cruise_control,keyless,price_for_large_families,exchange_with_bike,extra:826,right_sliding_door,exchange_boat_accepted,esp,interior_type,battery_range,seats,kteo,exterior_color,number_plate_ending,emissions_co2,battery_charge_time,interior_color,rim_size,vehicle_height,number_of_gears,torque,gross_weight,acceleration,vehicle_width,body_type,vehicle_length,top_speed,trim,wheelbase,fuel_consumption,drive_type,doors
0,41737621,Το αυτοκίνητο είναι σε υπέρ άριστη κατάσταση \nΌταν το πήρα για προληπτικούς λόγους και μόνο άλλαξα ολόκληρο σετ συμπλέκτη και φρένα \nΆριστο σε όλα του \n\n\nΠαρακαλώ όχι άσκοπα τηλεφωνήματα,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,None,None,5,None,Γκρι (Μεταλλικό),Μονή,None,None,Μαύρο,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,NaN,None,Προσθιοκίνητο (FWD),5
1,43324109,"• ΣΕ ΕΞΑΙΡΕΤΙΚΗ ΚΑΤΑΣΤΑΣΗ\n• BOOK SERVICE ΑΝΤΙΠΡΟΣΩΠΕΙΑΣ\n• ΕΝΤΟΣ ΕΡΓΟΣΤΑΣΙΑΚΗΣ ΕΓΓΥΗΣΗΣ\n• 1 ΧΕΡΙ, ΕΛΛΗΝΙΚO\n\n♠ 1.8 HSD E-CVT Business 122bhp ♠\n\n• ΔΕΡΜΑΤΙΝΟ ΤΙΜΟΝΙ ΜΕ ΧΕΙΡΙΣΤΗΡΙΑ\n• ΔΕΡΜΑΤΙΝΟ ΥΠΟΒΡΑΧΙΟΝΙΟ (Arm rest)\n• ΠΡΟΒΛΕΨΗ ΠΑΙΔΙΚΩΝ ΚΑΘΙΣΜΑΤΩΝ (Isofix)\n• ΥΠΟΛΟΓΙΣΤΗ ΤΑΞΙΔΙΟΥ (Trip computer)\n• ΨΗΦΙΑΚΗ ΟΘΟΝΗ ΠΟΛΛΑΠΛΩΝ ΕΝΔΕΙΞΕΩΝ\n• ΕΡΓΟΣΤΑΣΙΑΚΗ ΠΛΟΗΓΗΣΗ (Navigation)\n• ΚΑΜΕΡΑ ΟΠΙΣΘΟΠΟΡΕΙΑΣ (Reverse camera)\n• ΣΥΝΔΕΣΙΜΟΤΗΤΑ (Bluetooth/Usb/Aux)\n• ΨΗΦΙΑΚΟ ΚΛΙΜΑΤΙΣΜΟ (clima)\n• ΠΡΟΓΡΑΜΜΑ ΟΔΗΓΗΣΗΣ (Ev Mode)\n• ΑΥΤΟΜΑΤΟ ΠΙΛΟΤΟ (Cruise control)\n• ΑΙΣΘΗΤΗΡΑ ΦΩΤΩΝ (Light sensor)\n• ΑΙΣΘΗΤΗΡΑ ΒΡΟΧΗΣ (Rain sensor)\n• ΑΣΥΡΜΑΤΗ ΕΚΚΙΝΗΣΗ ΚΙΝΗΤΗΡΑ (Keyless GO)\n• ΣΥΣΤΗΜΑ ΑΝΤΙΜΠΛΟΚΑΡΙΣΜΑΤΟΣ ΤΡΟΧΩΝ (ABS)\n• ΗΛΕΚΤΡΟΝΙΚΟΣ ΕΛΕΓΧΟΣ ΕΥΣΤΑΘΕΙΑΣ (ESP)\n• ΖΑΝΤΕΣ ΑΛΟΥΜΙΝΙΟΥ 17""\n• ΦΩΤΑ ΗΜΕΡΑΣ (Led)\n\n- 1Η ΚΥΚΛΟΦΟΡΙΑ .10/10/2019\n• ΜΗΔΕΝΙΚΑ ΤΕΛΗ ΚΥΚΛΟΦΟΡΙΑΣ\n• VIN : NMTKZ3BX90R338616",False,False,False,False,True,True,False,True,True,False,False,False,True,False,False,False,False,False,True,True,False,False,True,True,False,True,False,False,True,False,True,True,False,False,False,False,True,False,False,False,True,True,False,True,False,True,False,False,True,True,True,False,True,False,True,True,True,True,True,True,True,True,True,False,True,True,True,False,False,False,False,True,True,False,False,True,True,True,True,False,True,False,False,False,True,Ύφασμα,None,5,None,Μαύρο (Μεταλλικό),Μονή,110 g/km,None,Γκρι,17 ίντσες,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,NaN,"3,80",Προσθιοκίνητο (FWD),5
2,43930480,None,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [368]:
(enrichment_df == False).sum()

id                      0
description             0
antique             87889
cheques_accepted    86455
turbo               47051
                    ...  
trim                    0
wheelbase               0
fuel_consumption        0
drive_type              0
doors                   0
Length: 111, dtype: int64